# KE6220 sweep programing example by the offical manual
## Configure source functions.
Examples – The following commands restore defaults, set the bias current to 100μA, and the compliance to 10V:

```
   *RST
   SOUR:CURR 1e-4
   SOUR:CURR:COMP 10
```

- ‘ Restore 622x defaults.
- ‘ Set bias current to 100μA
- ‘ Set compliance to 10V.


## Configure the sweep.

Examples – The following commands configure a single linear staircase sweep from 1mA to 10mA with 1mA steps using a 1s delay, best fixed source range, and compliance abort disabled:

```
   SOUR:SWE:SPAC LIN
   SOUR:CURR:STAR 1e-3
   SOUR:CURR:STOP 1e-2
   SOUR:CURR:STEP 1e-3
   SOUR:DEL 1
   SOUR:SWE:RANG BEST
   SOUR:SWE:COUN 1
   SOUR:SWE:CAB OFF
   
```

- ‘ Select linear staircase sweep. ‘ Set start current to 1mA.
- ‘ Set stop current to 10mA.
- ‘ Set step current to 1mA.
- ‘ Set delay to 1s.
- ‘ Select best fixed source range. ‘ Set sweep count to 1.
- ‘ Disable compliance abort.


## Arm and run the sweep:

```
SOUR:SWE:ARM
INIT
```
- ‘ Arm sweep, turn on output.
- ‘ Trigger sweep.

## When the sweep is done, turn the source output off with this command:

```
OUTP OFF
```


# Operation register for KE6220
(from C-lab "conductivity2.vi" block chart)

B0: CAL: Instrument Calibrating Event?  
B1: SwpD, Sweep, Delta, Diff Cond, or Pulse sweep Complete Event?  
B2: SwpA, Sweep, Delta, Diff Cond, or Pulse sweep Aborted Event?  
B3: Swp, Sweep, Delta, Diff Cond, or Pulse sweep Sweeping Event?  
B4: WStrt, KE6221 wave mode started Event?  
B5: Tring, Wating for trigger event event?  
B6: Arm, Waiting for Arm Event Event?  
B7: Wstop, KE6221 Wave Mode Aborted Event?  
B8: Filt, Delta, Test Started And Filter Settled Event?  
B9: none  
B10: Idle, Idle state Event?  
B11; RSE, RS-232 Error Event?  

My test shows that B3 =1 while sweeping, and 0 if it ends. This should be a good signal.

In [1]:
%matplotlib inline
from import_all import *


In [4]:
import visa
rm = visa.ResourceManager("/Library/Frameworks/Visa.framework/VISA")
print rm.list_resources()
K6220 = rm.open_resource('GPIB0::12::INSTR')
print(K6220.query("*IDN?"))

(u'GPIB0::7::INSTR', u'GPIB0::12::INSTR', u'GPIB0::13::INSTR', u'GPIB0::15::INSTR', u'GPIB0::22::INSTR')
KEITHLEY INSTRUMENTS INC.,MODEL 6220,1347935,A05  /700x 



In [66]:
import time
def simple_current_swp(bias_curr=0, cmpl=10,
                       curr_start=-1e-3, curr_stop=1e-3, curr_step=1e-4,
                       sour_del=1, swp_rang='BEST', swe_coun=1):
    
    ADD_TIME = 0.1 #[s] additional time for each step
    TIME_LAG = .5 #[s] a time lag betwee itinitaltion signal and sweep start
    
    step_num = (curr_stop - curr_start)/curr_step + 1
    meas_time = step_num * (sour_del + ADD_TIME)
    
    print 'est. meas time : %d [s]' %meas_time
    
    #basic settings
    inpt = K6220.write('*RST')
    inpt = K6220.write('SOUR:CURR ' + str(bias_curr))
    inpt = K6220.write('SOUR:CURR:COMP ' + str(cmpl))

    #setting for sweep 
    inpt = K6220.write('SOUR:SWE:SPAC LIN')
    inpt = K6220.write('SOUR:CURR:STAR ' + str(curr_start))
    inpt = K6220.write('SOUR:CURR:STOP ' + str(curr_stop))
    inpt = K6220.write('SOUR:CURR:STEP ' + str(curr_step))
    inpt = K6220.write('SOUR:DEL ' + str(sour_del))
    inpt = K6220.write('SOUR:SWE:RANG ' + swp_rang)
    inpt = K6220.write('SOUR:SWE:COUN ' + str(swe_coun))
    inpt = K6220.write('SOUR:SWE:CAB OFF')
    
    #execute sweep
    inpt = K6220.write('SOUR:SWE:ARM')
    inpt = K6220.write('INIT')
    
    time.sleep(TIME_LAG)

    for i in range(int(step_num) + 5):
        time.sleep(sour_del) 
        output = K6220.query('STAT:OPER:EVEN?')
        register = read_register(output, 12)
        #print i, register, register[-4]
        if register[-4]=='0': #if sweeping event(B4 = 0) is over
            print 'sweep completed'
            break

    #kill output
    inpt = K6220.write('OUTP OFF')
                       
    return 0
                       
def read_register(output, digit):
    out_enc = output.encode('utf-8')
    out_binary = str(format(int(out_enc), 'b'))
    out_string = out_binary.zfill(digit)
    return out_string



In [67]:
a = simple_current_swp()

est. meas time : 23 [s]
sweep completed
output:  



In [38]:
b = '010000001010'
print b[-4]

1
